<a href="https://colab.research.google.com/github/FabrizioSeverino/Portfolio/blob/main/Automation_Feature_DT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatizzare la selezione delle variabili con Python

La selezione delle variabili (feature selection) è una fase fondamentale nell’apprendimento automatico (machine learning), in quanto aiuta a migliorare le prestazioni del modello eliminando le caratteristiche irrilevanti o ridondanti.
Un metodo efficace per la selezione delle caratteristiche è l’uso degli Alberi Decisionali (Decision Trees), che classificano le variabili in base alla loro importanza nella previsione della variabile target.
In questo notebook, spiegherò come automatizzare la selezione delle caratteristiche con gli Alberi Decisionali usando Python.

**Cos'è la Selezione Automatica delle Caratteristiche?**  
La selezione automatica delle caratteristiche è come una pipeline che rimuove le caratteristiche irrilevanti o ridondanti, portando ai seguenti vantaggi:

- Tempi di addestramento più rapidi

- Riduzione dell’overfitting

- Maggiore accuratezza del modello

- Migliore interpretabilità del modello

La valutazione dell'importanza delle caratteristiche basata sugli Alberi Decisionali è uno dei metodi migliori per la selezione delle caratteristiche, poiché gli alberi decisionali assegnano naturalmente un punteggio di importanza a ciascuna variabile in base al loro contributo nella riduzione dell'errore.

Una compagnia di ride-sharing vuole implementare una strategia di prezzi dinamici per ottimizzare le tariffe in base alle condizioni di mercato in tempo reale. Attualmente, l’azienda utilizza solo la durata della corsa per determinare le tariffe. L'obiettivo è sfruttare tecniche basate sui dati per analizzare dati storici e sviluppare un modello predittivo in grado di regolare dinamicamente i prezzi in risposta ai cambiamenti delle condizioni.

È stato fornito un dataset contenente dati storici sulle corse. Il dataset include caratteristiche come:

- numero di passeggeri,

- numero di autisti,

- categoria della località,

- stato di fidelizzazione del cliente,

- numero di corse passate,

- valutazione media,

- orario della prenotazione,

- tipo di veicolo,

- durata prevista della corsa,

- costo storico delle corse.

Il mio obiettivo è costruire un modello di prezzo dinamico che incorpori le caratteristiche fornite per prevedere tariffe ottimali per le corse in tempo reale. Il modello deve tenere conto di fattori come i modelli di domanda e la disponibilità dell’offerta.


Il dataset è stato preso da Statso: [Dataset per l'esercitazione](https://statso.io/dynamic-pricing-case-study/)

Per prima cosa, importo i dati e analizzo la loro struttura per iniziare ad automatizzare la selezione delle variabili:

In [1]:
import pandas as pd

dataset = pd.read_csv('/content/dynamic_pricing.csv')
dataset.head()

,Number_of_Riders,Number_of_Drivers,Location_Category,Customer_Loyalty_Status,Number_of_Past_Rides,Average_Ratings,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,Historical_Cost_of_Ride
0,90,45,Urban,Silver,13,4.47,Night,Premium,90,284.257273
1,58,39,Suburban,Silver,72,4.06,Evening,Economy,43,173.874753
2,42,31,Rural,Silver,0,3.99,Afternoon,Premium,76,329.795469
3,89,28,Rural,Regular,67,4.31,Afternoon,Premium,134,470.201232
4,78,22,Rural,Regular,74,3.77,Afternoon,Economy,149,579.681422


La variabile target, Historical_Cost_of_Ride (costo della corsa), rappresenta il costo, mentre le altre variabili di input includono diversi valori numerici e categorici. Iniziamo ad automatizzare la selezione delle feature passo dopo passo.

**Passaggio 1: Codifica delle Variabili Categoriali**  
Useremo **LabelEncoder** per convertire le variabili categoriche in valori numerici:

In [4]:
from sklearn.preprocessing import LabelEncoder

# identifico le colonne categoriali
categorical_cols = ['Location_Category', 'Customer_Loyalty_Status', 'Time_of_Booking', 'Vehicle_Type']  # Replace with actual categorical columns

# applico il Label Encoding
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    dataset[col] = le.fit_transform(dataset[col])
    label_encoders[col] = le

Ora tutte le caratteristiche sono numeriche e pronte per la selezione delle feature.

**Passaggio 2: Addestramento di un albero decisionale per la selezione delle feature**  
Ora addestreremo un Decision Tree Regressor ed estrarremo l’importanza delle feature:

In [5]:
X = dataset.drop(columns=['Historical_Cost_of_Ride'])
y = dataset['Historical_Cost_of_Ride']

from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(random_state=42)
model.fit(X, y)

feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.feature_importances_
})

# Ordino per importanza
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

print(feature_importances)

                   Feature  Importance
8   Expected_Ride_Duration    0.877188
5          Average_Ratings    0.033423
4     Number_of_Past_Rides    0.021423
1        Number_of_Drivers    0.019312
0         Number_of_Riders    0.017676
7             Vehicle_Type    0.013145
2        Location_Category    0.009164
6          Time_of_Booking    0.005836
3  Customer_Loyalty_Status    0.002833


Informazione chiave: **Expected_Ride_Duration** è di gran lunga la caratteristica più importante.

**Passaggio 3: Automatizzare la selezione delle feature**  
Ora possiamo automatizzare la selezione delle feature mantenendo solo quelle con un’importanza superiore all’1%:

In [6]:
# definisco la soglia
soglia = 0.01  # Mantenere le features con un'importanza > 1%

features_selezionate = feature_importances[feature_importances['Importance'] > soglia]['Feature'].tolist()

# fltro il dataset
X_selected = X[features_selezionate]

Ora, **X_selected** contiene solo le caratteristiche più rilevanti per l’addestramento del modello finale. Ecco le migliori feature che abbiamo selezionato:

In [7]:
print(X_selected.columns)

Index(['Expected_Ride_Duration', 'Average_Ratings', 'Number_of_Past_Rides',
       'Number_of_Drivers', 'Number_of_Riders', 'Vehicle_Type'],
      dtype='object')


**Riepilogo**  
Un metodo efficace per la selezione delle caratteristiche è l’utilizzo degli Alberi Decisionali, che classificano le feature in base alla loro importanza nella previsione della variabile target.